#     **Handle multiple LLM models in Langchain and AWS Bedrock seamlessly**    ✅ 🔥  

## Pre-requisites
# To be able to advance through this tutorial, you will need:

A functional AWS account where you have admin permission .<br>
Have access to multiple LLM models on AWS Bedrock <br>
An AWS user setup on you laptop <br>
A functional Python environnement
Initialize the Jupyter lab
Let’s create the Jupyter lab setup. We will use pipenv to create a virtualenv for this project.<br> Creating a virtualenv is very important as it will make sure this project will run without harming your other ones.

Also please remember to always save your code using for example github.

      `Install pipenv`
Let’s begin by installing pipenv which will handle virtualenvs and python packages for us .

In [ ]:
!pip install pipenv

In [ ]:
!pipenv install langchain boto3

# Create the configurable multi LLM Langchain chat
Here we will create, using Langchain, a configurable LLM interface that can handle multiple LLM models in AWS Bedrock.

In [ ]:
from langchain_community.chat_models import BedrockChat
from langchain_core.runnables import ConfigurableField

#Here’s what is happening in this code:

1. We import the BedrockChat Langchain component that allows to use the AWS Bedrock models.
2. We also import ConfigurableField which will allow us to make this component configurable..

In [ ]:
aws_region_name = "us-west-2"
credentials_profile_name = "default"
claude_3_sonnet = "anthropic.claude-3-sonnet-20240229-v1:0"
mistral_large = "mistral.mistral-large-2402-v1:0"
mistral_large_bedrock_chat = BedrockChat(
    model_id=mistral_large,
    credentials_profile_name=credentials_profile_name,
    region_name=aws_region_name,
)
_model_alternatives = {
    "mistral_large": mistral_large_bedrock_chat
}

claude_3_sonnet = BedrockChat(
    model_id=claude_3_sonnet,
    credentials_profile_name=credentials_profile_name,
    region_name=aws_region_name,
)
bedrock_llm = claude_3_sonnet.configurable_alternatives(
    which=ConfigurableField(
        id="model", name="Model", description="The model that will be used"
    ),
    default_key="claude_3_sonnet",
    **_model_alternatives,
)

# *we create the configurable LLM interface:*

1. We put the claude_3_sonnet as the default one.
2. We added an alternative which is the mistral_large model.
3. We created the key model which will allow us to change the model we use.

In [ ]:
bedrock_llm.invoke("who are you ?")

bedrock_llm \
    .with_config(configurable={"model": "mistral_large"}) \
    .invoke("who are you ?")

**The BedrockChat component to use the Claude 3 Sonnet and Mistral Large models.**
# The _model_alternatives that will allows use to create a configurable LLM interface.

In [ ]:
from langchain_core.prompts import PromptTemplate

_MISTRAL_PROMPT = PromptTemplate.from_template(
    """
<s>[INST] You are a conversational AI designed to answer in a friendly way to a question.
You should always answer in rhymes.

Human:
<human_reply>
{input}
</human_reply>

Generate the AI's response.[/INST]</s>
"""
)

_CLAUDE_PROMPT = PromptTemplate.from_template(
    """
You are a conversational AI designed to answer in a friendly way to a question.
You should always answer in jokes.

Human:
<human_reply>
{input}
</human_reply>

Assistant:
"""
)

_CHAT_PROMPT_ALTERNATIVES = {"mistral_large": _MISTRAL_PROMPT}

We import and create PromptTemplate for Mistral and Claude which take into <br> account specificity of each models.<br>  For example, the Mistral model wants s [INST] inside the prompt.
We create two really different prompts to show how powerful this system is as you could have as many prompt alternative as you want and choose them at invoke time.
We create and alternative prompt which is the Mistral large one

In [ ]:
CONFIGURABLE_CHAT_PROMPT = _CLAUDE_PROMPT.configurable_alternatives(
    which=ConfigurableField(
        id="model",
        name="Model",
        description="The model that will be used",
    ),
    default_key="claude_3_sonnet",
    **_CHAT_PROMPT_ALTERNATIVES
)


In [ ]:
from langchain.schema.output_parser import StrOutputParser

chain = CONFIGURABLE_CHAT_PROMPT | bedrock_llm | StrOutputParser()

chain.invoke(input="What is a large language model ?")

bedrock_llm \
    .with_config(configurable={"model": "mistral_large"}) \
    .invoke("What is a large language model ?")